In [ ]:
# Makes the province themselves using an iterative Voronoi approach

import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

import numpy as np
from numba import cuda
from tqdm.notebook import tqdm

In [ ]:
def kernelCopyArray(limsy, limsx):
    @cuda.jit
    def f(popseeds, result):
        posy, posx = cuda.grid(2)
        
        if posy < limsy and posx < limsx: # Must have or crashes out of bounds
            popseeds[posy,posx] = result[posy,posx]
    return f
def kernelReduceToPoint(limsy, limsx):
    @cuda.jit
    def f(popseeds, result):
        posy, posx = cuda.grid(2)
        
        if posy < limsy and posx < limsx: # Must have or crashes out of bounds
            if popseeds[posy, posx]==0: return

            count = 0
            if posy-1>=0:
                if popseeds[posy-1, posx]!=0: count+=1
            if posx-1>=0:
                if popseeds[posy, posx-1]!=0: count+=1
            if posy<limsy:
                if popseeds[posy+1, posx]!=0: count+=1
            if posx<limsx:
                if popseeds[posy, posx+1]!=0: count+=1

            if count==2 or count==1:
                result[posy,posx] = 0
    return f

def reduceToPointGPU(popseeds):
    limsy, limsx = popseeds.shape

    gpuPopseeds = cuda.to_device(np.uint8(popseeds))
    gpuResult = cuda.to_device(np.uint8(popseeds))

    threadsperblock = (16, 16)
    blockspergridy = np.ceil(limsy / threadsperblock[0]).astype(np.int32)
    blockspergridx = np.ceil(limsx / threadsperblock[1]).astype(np.int32)
    blockspergrid = (blockspergridy, blockspergridx)

    for _ in range(10):
        kernelReduceToPoint(limsy, limsx)[blockspergrid, threadsperblock](gpuPopseeds, gpuResult)
        kernelCopyArray(limsy, limsx)[blockspergrid, threadsperblock](gpuPopseeds, gpuResult)

    popseeds = gpuPopseeds.copy_to_host()

    # Deallocation
    gpuPopseeds, gpuResult = None, None
    return popseeds

In [ ]:
# Import water mask and image with gray-128 circles indicating where the province seeds lay. It's important that the circles have a unique center pixel, which usually means uneven brush width like 15 or 5
popseeds = np.array(Image.open('intermediate/iterativeVoronoi/popseeds.png').convert('L'))
seaMask = np.array(Image.open('intermediate/seaMask.png').convert('L'))
riverMask = np.array(Image.open('intermediate/lakeRiverCenterlines.png').convert('L'))
impassableMask = np.array(Image.open('intermediate/impassablesMask.png').convert('L'))
lakeMask = np.array(Image.open('intermediate/waterNoRivers.png').convert('L'))
lakeMask[seaMask==255] = 0

# Reduce circles into isolated pixels
popseeds = np.uint16(reduceToPointGPU(popseeds))
#Image.fromarray(np.uint8(popseeds/np.max(popseeds)*255)).save('testtttt.png')

# Build process colors,                         # 0: available seas
popseeds[(popseeds==0) & (seaMask==0)] = 1      # 1: available land
popseeds[riverMask==255] = 2                      # 2: rivers
popseeds[lakeMask==255] = 3                     # 3: lakes
popseeds[impassableMask==255] = 10            # 10 for impassables

# Find out how many provinces we have by counting those pixels
nSeaProvinces = np.sum((popseeds==76) & (seaMask==255))
nLandProvinces = np.sum((popseeds==150) & (seaMask==0))
nSpecialProvinces = np.sum(popseeds==203)
nProvinces = nSeaProvinces + nLandProvinces + nSpecialProvinces

# Generate unique colors for each id
colors = {
    'sea': set(),
    'land': set(),
    'special': set()
}
while len(colors['sea']) < nSeaProvinces:
    color = tuple(np.uint8(x) for x in [1]+list(np.random.randint(1, 255, size=1))+list(np.random.randint(150, 255, size=1)))
    colors['sea'].add(color)
colors['sea'] = list(colors['sea'])
while len(colors['land']) < nLandProvinces:
    color = tuple(np.uint8(x) for x in list(np.random.randint(55, 255, size=2))+[1])
    colors['land'].add(color)
colors['land'] = list(colors['land'])
while len(colors['special']) < nSpecialProvinces:
    color = tuple(np.uint8(x) for x in list(np.random.randint(180, 255, size=2))+[100])
    colors['special'].add(color)
colors['special'] = list(colors['special'])

# Assign id to each dot. The first 10 are reserved for the build process
locations = np.where((popseeds==76) | (popseeds==150) | (popseeds==203))
countSea = 0
countLand = 0
countSpecial = 11
for i in tqdm(range(len(locations[0]))):
    y, x = locations[0][i], locations[1][i]
    if popseeds[y,x] == 203:
        popseeds[y,x] = countSpecial
        countSpecial += 1
    elif seaMask[y,x] == 0:
        popseeds[y,x] = countLand+nSpecialProvinces+11
        countLand += 1
    elif seaMask[y,x] == 255:
        popseeds[y,x] = countSea+nLandProvinces+nSpecialProvinces+11
        countSea += 1

print(f'Provinces: {nProvinces} ({nSeaProvinces} sea / {nLandProvinces} land / {nSpecialProvinces} special)')

In [ ]:
def kernelPaintFromId(limsy, limsx, nSpecialProvinces, nLandProvinces):
    @cuda.jit
    def f(provinces, result, colorSpecial, colorLand, colorSea):
        posy, posx = cuda.grid(2)
        
        if posy < limsy and posx < limsx: # Must have or crashes out of bounds
            ids = provinces[posy, posx]
            if ids>10:
                if ids>=nSpecialProvinces+nLandProvinces+11:
                    ids = int(ids-nSpecialProvinces-nLandProvinces-11)
                    color = colorSea[ids]
                elif ids>=nSpecialProvinces+11:
                    ids = int(ids-nSpecialProvinces-11)
                    color = colorLand[ids]
                else:
                    ids = int(ids-11)
                    color = colorSpecial[ids]
                result[posy,posx] = (color[0], color[1], color[2])
            elif ids==1:
                result[posy,posx] = (255,255,255)
            else:
                result[posy,posx] = (ids,ids,ids)
    return f

def paintFromIdGPU(provinces, colors, nSpecialProvinces, nLandProvinces):
    limsy, limsx = popseeds.shape

    gpuProvinces = cuda.to_device(provinces)
    gpuColorSpecial = cuda.to_device(np.array(colors['special']))
    gpuColorLand = cuda.to_device(np.array(colors['land']))
    gpuColorSea = cuda.to_device(np.array(colors['sea']))
    gpuResult = cuda.to_device(np.zeros((limsy,limsx,3), dtype=np.uint8))

    threadsperblock = (16, 16)
    blockspergridy = np.ceil(limsy / threadsperblock[0]).astype(np.int32)
    blockspergridx = np.ceil(limsx / threadsperblock[1]).astype(np.int32)
    blockspergrid = (blockspergridy, blockspergridx)

    kernelPaintFromId(limsy, limsx, nSpecialProvinces, nLandProvinces)[blockspergrid, threadsperblock](gpuProvinces, gpuResult, gpuColorSpecial, gpuColorLand, gpuColorSea)
    result = gpuResult.copy_to_host()
    
    # Deallocation
    gpuProvinces, gpuColorSpecial, gpuColorLand, gpuColorSea, gpuResult = None, None, None, None, None
    return result

In [ ]:
# # Run this at least once. It creates a few random images for the iterations
# for i in range(10):
#     np.save(f'intermediate/iterativeVoronoi/random/{i}.npy', np.float16(np.random.rand(8192, 16384)))

In [ ]:
import concurrent.futures
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
def kernelCopyArray(limsy, limsx):
    @cuda.jit
    def f(provinces, provincesRes, condition, conditionRes):
        y, x = cuda.grid(2)
        
        if y < limsy and x < limsx: # Must have or crashes out of bounds
            provinces[y,x] = provincesRes[y,x]
            condition[y,x] = conditionRes[y,x]
    return f
def kernelVoronoi(limsy, limsx, nSpecialProvinces, nLandProvinces):
    @cuda.jit
    def seaCheck(ids2):
        if ids2==0:
            return True
        return False
    @cuda.jit
    def landCheck(ids2):
        if ids2==1:
            return True
        return False
    @cuda.jit
    def specialCheck(ids2):
        if ids2==1 or ids2==2:
            return True
        return False
    @cuda.jit
    def applySingleChange(provinces, provincesRes, conditionRes, ids, y, x, neighborCheck):
        ids2 = provinces[y,x]
        if neighborCheck(ids2):
            provincesRes[y,x] = ids
            conditionRes[y,x] = True
    @cuda.jit
    def applyChanges(y, x, conditionRes, ids, provinces, provincesRes, neighborCheck):
        if y-1>=0: applySingleChange(provinces, provincesRes, conditionRes, ids, y-1, x, neighborCheck)
        if x-1>=0: applySingleChange(provinces, provincesRes, conditionRes, ids, y, x-1, neighborCheck)
        if y+1<limsy: applySingleChange(provinces, provincesRes, conditionRes, ids, y+1, x, neighborCheck)
        if x+1<limsx: applySingleChange(provinces, provincesRes, conditionRes, ids, y, x+1, neighborCheck)
    @cuda.jit
    def f(provinces, provincesRes, condition, conditionRes, randomStates):
        y, x = cuda.grid(2)
        
        if y < limsy and x < limsx: # Must have or crashes out of bounds
            if condition[y,x]==False: return
            randomState = randomStates[y,x]
            if randomState>0.5: return
            ids = provinces[y,x]
            if ids>10:
                if ids>nSpecialProvinces+nLandProvinces+10: # Sea
                    applyChanges(y, x, conditionRes, ids, provinces, provincesRes, seaCheck)
                elif ids>nSpecialProvinces+10: # Land
                    applyChanges(y, x, conditionRes, ids, provinces, provincesRes, landCheck)
                else: # Special
                    if randomState>0.2: return
                    applyChanges(y, x, conditionRes, ids, provinces, provincesRes, specialCheck)
                conditionRes[y,x] = False
    return f

def saveImage(coloredProvinces, i):
    Image.fromarray(coloredProvinces).save(f'intermediate/iterativeVoronoi/steps/{i:04d}.png')

def voronoiGPU(
        popseeds: np.ndarray,
        nSpecialProvinces: int,
        nLandProvinces: int,
        cycles: int,
        jumpOverLand: int,
        jumpOverWater: int,
        printImgs: bool=False
    ) -> np.ndarray:
    limsy, limsx = popseeds.shape

    gpuRandomNumbers = []
    for i in range(10): gpuRandomNumbers.append(cuda.to_device(np.load(f'intermediate/iterativeVoronoi/random/{i}.npy')))
    gpuRandomNumbers.append(cuda.to_device(np.zeros(shape=popseeds.shape, dtype=np.float16)))
    selectedRandomNumbers = 10

    gpuProvinces = cuda.to_device(popseeds)
    gpuProvincesRes = cuda.to_device(popseeds)
    gpuCondition = cuda.to_device(popseeds>10)
    gpuConditionRes = cuda.to_device(popseeds>10)

    gpuColorSpecial = cuda.to_device(np.array(colors['special']))
    gpuColorLand = cuda.to_device(np.array(colors['land']))
    gpuColorSea = cuda.to_device(np.array(colors['sea']))
    gpuColoredProvinces = cuda.to_device(np.zeros((limsy,limsx,3), dtype=np.uint8))

    threadsperblock = (32, 32)
    blockspergridy = np.ceil(limsy / threadsperblock[0]).astype(np.int32)
    blockspergridx = np.ceil(limsx / threadsperblock[1]).astype(np.int32)
    blockspergrid = (blockspergridy, blockspergridx)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for i in tqdm(range(cycles)):
            if (i % 5 == 0) or (i<5):
                if i!=0:
                    selectedRandomNumbers = np.random.randint(0,10)
                
            kernelVoronoi(limsy, limsx, nSpecialProvinces, nLandProvinces)[blockspergrid, threadsperblock](
                gpuProvinces, 
                gpuProvincesRes, 
                gpuCondition, 
                gpuConditionRes, 
                gpuRandomNumbers[selectedRandomNumbers]
            )
            if i == jumpOverLand:
                frozenMap = gpuProvincesRes.copy_to_host()
                provinces = frozenMap.copy()
                provinces[popseeds > 0] = 0
                gpuProvincesRes = cuda.to_device(provinces)
                gpuConditionRes = cuda.to_device(provinces > 10)
            elif i == jumpOverWater:
                provinces = gpuProvincesRes.copy_to_host()
                provinces[popseeds != 0] = frozenMap[popseeds != 0]
                frozenMap = provinces.copy()
                provinces = frozenMap.copy()
                provinces[popseeds == 0] = 1
                provinces[provinces < 11] = 1
                gpuProvincesRes = cuda.to_device(provinces)
                gpuConditionRes = cuda.to_device(provinces > 10)
            elif (i==cycles-1) and (i>jumpOverWater):
                provinces = gpuProvincesRes.copy_to_host()
                popseeds[popseeds == 0] = frozenMap[popseeds == 0]
                popseeds[popseeds == 1] = provinces[popseeds == 1]
                gpuProvincesRes = cuda.to_device(popseeds)
            
            kernelCopyArray(limsy, limsx)[blockspergrid, threadsperblock](gpuProvinces, gpuProvincesRes, gpuCondition, gpuConditionRes)
            # This copy function is fundamental. Simultaneously using and writing the provinces and conditions arrays were giving very strange memory errors. We need different arrays for writing vs reading

            if printImgs and (i % 5 == 0):
                kernelPaintFromId(limsy, limsx, nSpecialProvinces, nLandProvinces)[blockspergrid, threadsperblock](gpuProvinces, gpuColoredProvinces, gpuColorSpecial, gpuColorLand, gpuColorSea)
                coloredProvinces = gpuColoredProvinces.copy_to_host()
                # Submit the image saving task to the executor
                executor.submit(saveImage, coloredProvinces.copy(), i)

            

        
    gpuProvinces.copy_to_host(provinces)
        
    # Deallocation
    gpuProvinces, gpuRandomNumbers = None, None
    return provinces

In [ ]:
provinces = voronoiGPU(popseeds, nSpecialProvinces, nLandProvinces, cycles=901, jumpOverLand=600, jumpOverWater=650, printImgs=True)

# coloredProvinces = paintFromIdGPU(provinces, colors, nSpecialProvinces, nLandProvinces)
# Image.fromarray(coloredProvinces).show()